In [1]:
#Constants:
c_train_df_prep_file='train_df_prep_file.pkl'
c_bst_file='bst.pkl'
c_accuracy='c_accuracy.csv'

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import lightgbm as lgb
import gc

In [3]:
metrics = 'auc'

In [4]:
lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric':metrics,
        'learning_rate': 0.1,
        'num_leaves': 7,  # we should let it be smaller than 2^(max_depth)
        'max_depth': 4,  # -1 means no limit
        'min_child_samples': 100,  # Minimum number of data need in a child(min_data_in_leaf)
        'max_bin': 100,  # Number of bucketed bin for feature values
        'subsample': 0.7,  # Subsample ratio of the training instance.
        'subsample_freq': 1,  # frequence of subsample, <=0 means no enable
        'colsample_bytree': 0.7,  # Subsample ratio of columns when constructing each tree.
        'min_child_weight': 0,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
        'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
        'nthread': 4,
        'verbose': 0,
        'scale_pos_weight':99.7, # because training data is extremely unbalanced 
#         'metric':metrics
}
target = 'is_attributed'
predictors = ['app','device','os', 'channel', 'hour','nip_day_test_hh', 'nip_day_hh',
              'nip_hh_os', 'nip_hh_app', 'nip_hh_dev']
categorical = ['app', 'device', 'os', 'channel', 'hour']

In [5]:
train_df_prep=pd.read_pickle(c_train_df_prep_file)
train_df, val_df = train_test_split( train_df_prep, train_size=.95, random_state=99, shuffle=False )
 
print(train_df.info())
print(val_df.info())

print("train size: ", len(train_df))
print("valid size: ", len(val_df))
del train_df_prep
gc.collect()

FileNotFoundError: [Errno 2] No such file or directory: 'train_df_prep_file.pkl'

In [ ]:
VALIDATE = True
MAX_ROUNDS = 500
EARLY_STOP = 50
OPT_ROUNDS = 500
print("Training...")

num_boost_round=MAX_ROUNDS
early_stopping_rounds=EARLY_STOP

xgtrain = lgb.Dataset(train_df[predictors].values, label=train_df[target].values,
                      feature_name=predictors,
                      categorical_feature=categorical
                      )
del train_df
gc.collect()
print('done...')

In [ ]:
xgvalid = lgb.Dataset(val_df[predictors].values, label=val_df[target].values,
                      feature_name=predictors,
                      categorical_feature=categorical
                      )
del val_df
gc.collect()

In [6]:
lgb_params

{'boosting_type': 'gbdt',
 'colsample_bytree': 0.7,
 'learning_rate': 0.1,
 'max_bin': 100,
 'max_depth': 4,
 'metric': 'auc',
 'min_child_samples': 100,
 'min_child_weight': 0,
 'min_split_gain': 0,
 'nthread': 4,
 'num_leaves': 7,
 'objective': 'binary',
 'scale_pos_weight': 99.7,
 'subsample': 0.7,
 'subsample_freq': 1,
 'verbose': 0}

In [7]:
evals_results = {}
bst = lgb.train(lgb_params, 
                 xgtrain, 
                 valid_sets=[xgtrain, xgvalid], 
                 valid_names=['train','valid'], 
                 evals_result=evals_results, 
                 num_boost_round=num_boost_round,
                 early_stopping_rounds=early_stopping_rounds,
                 verbose_eval=10, 
                 feval=None)

n_estimators = bst.best_iteration
bst.save_model(c_bst_file)
print("\nModel Report")
print("n_estimators : ", n_estimators)
# print(metrics+":", evals_results['valid'][metrics][n_estimators-1])
del xgvalid

NameError: name 'xgtrain' is not defined

In [8]:
evals_results['valid'][metrics][n_estimators-1]

KeyError: 'valid'

In [9]:

import datetime
date_time=datetime.datetime.now()
train_acc=evals_results['train']['auc'][n_estimators-1]
valid_acc=evals_results['valid']['auc'][n_estimators-1]
bst_iter=n_estimators
comment='Trained with all data, 5 features from andys'

KeyError: 'train'

In [10]:
#prepare metrics and see
col_list=['date_time','train_acc','valid_acc','best_iter','cols','comment']
curr_data=pd.DataFrame([[date_time,train_acc,valid_acc,bst_iter,predictors,comment]], columns=col_list)
curr_data.head()

NameError: name 'train_acc' is not defined

,date_time,train_acc,valid_acc,best_iter,comment
0,2018-04-09 13:53:44.416075,0.985568,0.96775,61,"Trained with all data, 5 features from andys"
